In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier



url = '/Users/Roby/MLFieldWork/DF_Model.csv'
target = 'target'
variables = ['client_create_date', 'region', 'trade_sector', 'n_employees', 'economic_pot', 'eco_pot_class', 'risk_cat', 'flg_tool', 'sales_channel', 'net', 'unit', 'family_code', 'group_code', 'n_purchases', 'sales_net']


In [26]:
df = pd.read_csv(url)
df.head()

,client_id,client_create_date,region,trade_sector,n_employees,economic_pot,eco_pot_class,risk_cat,yyyymm,item_id,...,sales_channel,net,unit,family_code,group_code,sales_id,n_purchases,sales_net,time_diff,target
0,9306,2005-11-15,BZ,11000,6,8659.81,D,3d,2017-09-01,12585,...,C,937.94,True,XBXV2EB,XBXV2EB0102,9306_201709,2.0,1579.03,NaN,1
1,9306,2005-11-15,BZ,11000,6,8659.81,D,3d,2017-09-01,13431,...,C,641.09,True,XBXV2AB,XBXV2AB0102,9306_201709,2.0,1579.03,NaN,1
2,9306,2005-11-15,BZ,11000,6,8659.81,D,3d,2017-09-01,1380,...,C,0.00,True,XBXV2EB,XBXV2EB0102,9306_201709,2.0,1579.03,NaN,1
3,9306,2005-11-15,BZ,11000,6,8659.81,D,3d,2020-10-01,3493,...,C,1341.14,True,XBXV1AF,XBXV1AF0201,9306_202010,2.0,1341.14,1126.0,1
4,8321,2005-11-15,LE,15500,2,681.26,E,T8,2018-04-01,6776,...,B,20.02,True,XBSI4AA,XBSI4AA0101,8321_201804,7.0,20.02,NaN,1


In [52]:
df.columns

Index(['client_id', 'client_create_date', 'region', 'trade_sector',
       'n_employees', 'economic_pot', 'eco_pot_class', 'risk_cat', 'yyyymm',
       'item_id', 'flg_tool', 'sales_channel', 'net', 'unit', 'family_code',
       'group_code', 'sales_id', 'n_purchases', 'time_diff', 'target',
       'sales_net'],
      dtype='object')

In [37]:
df.drop('sales_net', axis=1, inplace=True)

sales_net = df.groupby('sales_id')['net'].sum()
sales_net = sales_net.reset_index()
sales_net.columns = ['sales_id', 'sales_net']
df = pd.merge(df, sales_net, on='sales_id')

df[['client_id', 'item_id', 'sales_id', 'trade_sector']] = df[['client_id', 'item_id', 'sales_id', 'trade_sector']].astype(str)

df

,client_id,client_create_date,region,trade_sector,n_employees,economic_pot,eco_pot_class,risk_cat,yyyymm,item_id,...,sales_channel,net,unit,family_code,group_code,sales_id,n_purchases,time_diff,target,sales_net
0,9306,2005-11-15,BZ,11000,6,8659.81,D,3d,2017-09-01,12585,...,C,937.94,True,XBXV2EB,XBXV2EB0102,9306_201709,2.0,NaN,1,1579.03
1,9306,2005-11-15,BZ,11000,6,8659.81,D,3d,2017-09-01,13431,...,C,641.09,True,XBXV2AB,XBXV2AB0102,9306_201709,2.0,NaN,1,1579.03
2,9306,2005-11-15,BZ,11000,6,8659.81,D,3d,2017-09-01,1380,...,C,0.00,True,XBXV2EB,XBXV2EB0102,9306_201709,2.0,NaN,1,1579.03
3,9306,2005-11-15,BZ,11000,6,8659.81,D,3d,2020-10-01,3493,...,C,1341.14,True,XBXV1AF,XBXV1AF0201,9306_202010,2.0,1126.0,1,1341.14
4,8321,2005-11-15,LE,15500,2,681.26,E,T8,2018-04-01,6776,...,B,20.02,True,XBSI4AA,XBSI4AA0101,8321_201804,7.0,NaN,1,20.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421036,93244,2021-12-23,RE,11000,1,1509.51,E,5d,2021-12-01,12174,...,B,0.00,True,XADC1AA,XADC1AA0401,93244_202112,1.0,NaN,0,7365.95
421037,93244,2021-12-23,RE,11000,1,1509.51,E,5d,2021-12-01,2841,...,B,331.40,True,XADD2EB,XADD2EB0201,93244_202112,1.0,NaN,0,7365.95
421038,93244,2021-12-23,RE,11000,1,1509.51,E,5d,2021-12-01,5292,...,B,2298.99,True,XADD1BA,XADD1BA0602,93244_202112,1.0,NaN,0,7365.95
421039,93244,2021-12-23,RE,11000,1,1509.51,E,5d,2021-12-01,5282,...,B,1399.00,True,XADD1BB,XADD1BB0401,93244_202112,1.0,NaN,0,7365.95


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421041 entries, 0 to 421040
Data columns (total 21 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   client_id           421041 non-null  object 
 1   client_create_date  421041 non-null  object 
 2   region              412144 non-null  object 
 3   trade_sector        421041 non-null  object 
 4   n_employees         421041 non-null  int64  
 5   economic_pot        421041 non-null  float64
 6   eco_pot_class       421041 non-null  object 
 7   risk_cat            421041 non-null  object 
 8   yyyymm              421041 non-null  object 
 9   item_id             421041 non-null  object 
 10  flg_tool            421041 non-null  int64  
 11  sales_channel       421041 non-null  object 
 12  net                 421041 non-null  float64
 13  unit                421041 non-null  bool   
 14  family_code         421041 non-null  object 
 15  group_code          421041 non-nul

In [56]:
# Select numerical columns for SelectKBest
numerical_features = df.select_dtypes(include=['int64', 'float64']).columns.drop(['target', 'time_diff'])

# Apply SelectKBest
X = df[numerical_features]
y = df[target]


selector = SelectKBest(score_func=f_classif, k='all')  # Select all features for ranking
X_selected = selector.fit(X, y)

# Show feature scores
feature_scores = pd.DataFrame({'Feature': numerical_features, 'Score': X_selected.scores_})
print(feature_scores.sort_values(by='Score', ascending=False))

        Feature         Score
4   n_purchases  82158.301438
1  economic_pot   7598.385969
5     sales_net   1830.225116
2      flg_tool     86.872493
0   n_employees     40.948012
3           net     15.951902


In [57]:
# Train a Random Forest model
rf = RandomForestClassifier(n_estimators=100, random_state=42)
X = df[variables]
y = df[target]
rf.fit(X, y)

# Get feature importances
feature_importance = pd.DataFrame({'Feature': numerical_features, 'Importance': rf.feature_importances_})
print(feature_importance.sort_values(by='Importance', ascending=False))

ValueError: could not convert string to float: '2005-11-15'